# Personal Assistant Testing

Test the email assistant with HITL and memory using deepagents.

In [36]:
from personal_assistant import create_email_assistant
from personal_assistant.ntbk_utils import format_messages, show_prompt

## Setup

Create the email assistant agent.

In [37]:
# Create agent
agent = create_email_assistant()
print("✓ Agent created successfully")

✓ Agent created successfully


## Example 1: Meeting Request

Test the agent with a meeting request email.

In [38]:
# Helper functions for testing
from langgraph.types import Command

def resume_with_approve_or_reject(agent, config, decision_type="approve", message=None):
    """Resume agent with approve or reject decision.
    
    Args:
        agent: The agent instance
        config: Config with thread_id
        decision_type: "approve" or "reject"
        message: Optional rejection message with feedback
    """
    if decision_type == "approve":
        decision = {"type": "approve"}
    elif decision_type == "reject":
        # Include optional rejection message
        decision = {"type": "reject", "message": message if message else "User rejected this action"}
    else:
        raise ValueError(f"Invalid decision type: {decision_type}")
    
    # Resume with decisions wrapped in dict (required by built-in HITL middleware)
    result = agent.invoke(
        Command(resume={"decisions": [decision]}),
        config=config,
    )
    return result

def get_user_profile_from_store(agent):
    """Get the current user profile from the agent's store."""
    # Access the store from the agent's compiled graph
    store = agent.store
    namespace = ("email_assistant", "user_profile")
    key = "user_preferences"
    
    # Get the item from store
    item = store.get(namespace, key)
    if item:
        return item.value
    return None
 
print("✓ Helper functions loaded")

✓ Helper functions loaded


In [39]:
# Example email as markdown string
email_markdown = """
**Subject**: Quick question about next week
**From**: jane@example.com
**To**: lance@langchain.dev

Hi Lance,

Can we meet next Tuesday at 2pm to discuss the project roadmap?

Best,
Jane

---
"""

# Display email with rich formatting
show_prompt(email_markdown, title="📧 Email to Process", border_style="cyan")

╭────────────────────────────────────────────── 📧 Email to Process ──────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  **Subject**: Quick question about next week                                                                    │
│  **From**: jane@example.com                                                                                     │
│  **To**: lance@langchain.dev                                                                                    │
│                                                                                                                 │
│  Hi Lance,                                                                                                      │
│                                                                                                                 │
│  Can we meet next Tuesday at 2pm to discuss the project roadmap?                                                │
│                                                                                                                 │
│  Best,                                                                                                          │
│  Jane                                                                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [40]:
# Configure thread
config_1 = {"configurable": {"thread_id": "test-thread-1"}}

# Invoke agent with markdown email string
result_1 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown}]},
    config=config_1,
)

# Display result with rich formatting
print("\n" + "="*60)
print("Agent Response")
print("="*60 + "\n")

# Check for interrupts (HITL)
if "__interrupt__" in result_1:
    print("🛑 INTERRUPT: Agent is waiting for your approval\n")
    
    for interrupt in result_1["__interrupt__"]:
        # New interrupt format with action_requests
        action_requests = interrupt.value.get('action_requests', [])
        review_configs = interrupt.value.get('review_configs', [])
        
        for action_req in action_requests:
            print("Action:", action_req['name'])
            print("Args:", action_req['args'])
            print("\nDescription:")
            show_prompt(action_req.get('description', ''), title="📋 Action Details", border_style="yellow")
            print("\n" + "-"*60)
        
        if review_configs:
            print("\nAllowed decisions:", review_configs[0].get('allowed_decisions', []))
    
    print("\n💡 To continue, you need to resume the agent with a decision.")
    print("   Use: resume_with_approve_or_reject(agent, config_1, decision_type='approve')")
else:
    format_messages(result_1["messages"])


Agent Response

🛑 INTERRUPT: Agent is waiting for your approval

Action: schedule_meeting
Args: {'attendees': ['jane@example.com'], 'subject': 'Project Roadmap Discussion', 'duration_minutes': 30, 'preferred_day': '2025-12-10T14:00:00', 'start_time': 14}

Description:


╭─────────────────────────────────────────────── 📋 Action Details ───────────────────────────────────────────────╮
│                                                                                                                 │
│  I've prepared a calendar invitation. Please review the meeting details below. Approve to send the invite       │
│  as-is, or Reject to cancel and end the workflow.                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


------------------------------------------------------------

Allowed decisions: ['approve', 'reject']

💡 To continue, you need to resume the agent with a decision.
   Use: resume_with_approve_or_reject(agent, config_1, decision_type='approve')


### Example: Resume by approving the action

**Note:** The new interrupt system uses `resume_with_approve_or_reject()` with "approve" or "reject" decisions only.

In [41]:
result_1_continued = resume_with_approve_or_reject(agent, config_1, decision_type="approve")
format_messages(result_1_continued["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ **Subject**: Quick question about next week                                                                     │
│ **From**: jane@example.com                                                                                      │
│ **To**: lance@langchain.dev                                                                                     │
│                                                                                                                 │
│ Hi Lance,                                                                                                       │
│                                                                                                                 │
│ Can we meet next Tuesday at 2pm to discuss the project roadmap?                                                 │
│                                                                                                                 │
│ Best,                                                                                                           │
│ Jane                                                                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: triage_email                                                                                      │
│    Args: {                                                                                                      │
│   "reasoning": "This email is from Jane asking to meet next Tuesday at 2pm to discuss the project roadmap. This │
│ is a direct meeting request with a specific time proposed. According to the user profile, meeting requests      │
│ should be classified as 'respond'. The email requires a response to either confirm the meeting time or propose  │
│ alternatives.",                                                                                                 │
│   "classification": "respond"                                                                                   │
│ }                                                                                                               │
│    ID: toolu_01BkfgaUTdJgcpawMwqTMeYr                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Classification Decision: respond. Reasoning: This email is from Jane asking to meet next Tuesday at 2pm to      │
│ discuss the project roadmap. This is a direct meeting request with a specific time proposed. According to the   │
│ user profile, meeting requests should be classified as 'respond'. The email requires a response to either       │
│ confirm the meeting time or propose alternatives.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Now I need to check calendar availability for next Tuesday (December 10, 2025) to see if 2pm is available.      │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-10"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_01WNeCsBFLNnK34k7A5QrtNv                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-10: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Great! 2pm is available on Tuesday, December 10th. Now I'll schedule the meeting for 30 minutes (as per the     │
│ user profile preference).                                                                                       │
│                                                                                                                 │
│ 🔧 Tool Call: schedule_meeting                                                                                  │
│    Args: {                                                                                                      │
│   "attendees": [                                                                                                │
│     "jane@example.com"                                                                                          │
│   ],                                                                                                            │
│   "subject": "Project Roadmap Discussion",                                                                      │
│   "duration_minutes": 30,                                                                                       │
│   "preferred_day": "2025-12-10T14:00:00",                                                                       │
│   "start_time": 14                                                                                              │
│ }                                                                                                               │
│    ID: toolu_01AsrXaBC6K3At2YNzBJ7wSg                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Meeting 'Project Roadmap Discussion' scheduled on Wednesday, December 10, 2025 at 14 for 30 minutes with 1      │
│ attendees                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Perfect! The meeting has been scheduled. Now I'll send a confirmation email to Jane.                            │
│                                                                                                                 │
│ 🔧 Tool Call: write_email                                                                                       │
│    Args: {                                                                                                      │
│   "to": "jane@example.com",                                                                                     │
│   "subject": "Re: Quick question about next week",                                                              │
│   "content": "Hi Jane,\n\nTuesday at 2pm works perfectly. I've scheduled 30 minutes for us to discuss the       │
│ project roadmap.\n\nLooking forward to it!\n\nBest,\nLance"                                                     │
│ }                                                                                                               │
│    ID: toolu_01WUudqPrLWfUb2wGZDorAHz                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Example: Resume by rejecting 

The interrupt system uses "reject" Rejecting will update the user profile to learn from your feedback.

In [42]:
result_1_continued = resume_with_approve_or_reject(agent, config_1, decision_type="reject", message="Not interested in this meeting")
format_messages(result_1_continued["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ **Subject**: Quick question about next week                                                                     │
│ **From**: jane@example.com                                                                                      │
│ **To**: lance@langchain.dev                                                                                     │
│                                                                                                                 │
│ Hi Lance,                                                                                                       │
│                                                                                                                 │
│ Can we meet next Tuesday at 2pm to discuss the project roadmap?                                                 │
│                                                                                                                 │
│ Best,                                                                                                           │
│ Jane                                                                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: triage_email                                                                                      │
│    Args: {                                                                                                      │
│   "reasoning": "This email is from Jane asking to meet next Tuesday at 2pm to discuss the project roadmap. This │
│ is a direct meeting request with a specific time proposed. According to the user profile, meeting requests      │
│ should be classified as 'respond'. The email requires a response to either confirm the meeting time or propose  │
│ alternatives.",                                                                                                 │
│   "classification": "respond"                                                                                   │
│ }                                                                                                               │
│    ID: toolu_01BkfgaUTdJgcpawMwqTMeYr                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Classification Decision: respond. Reasoning: This email is from Jane asking to meet next Tuesday at 2pm to      │
│ discuss the project roadmap. This is a direct meeting request with a specific time proposed. According to the   │
│ user profile, meeting requests should be classified as 'respond'. The email requires a response to either       │
│ confirm the meeting time or propose alternatives.                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Now I need to check calendar availability for next Tuesday (December 10, 2025) to see if 2pm is available.      │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-10"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_01WNeCsBFLNnK34k7A5QrtNv                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-10: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Perfect! The meeting has been scheduled. Now I'll send a confirmation email to Jane.                            │
│                                                                                                                 │
│ 🔧 Tool Call: write_email                                                                                       │
│    Args: {                                                                                                      │
│   "to": "jane@example.com",                                                                                     │
│   "subject": "Re: Quick question about next week",                                                              │
│   "content": "Hi Jane,\n\nTuesday at 2pm works perfectly. I've scheduled 30 minutes for us to discuss the       │
│ project roadmap.\n\nLooking forward to it!\n\nBest,\nLance"                                                     │
│ }                                                                                                               │
│    ID: toolu_01WUudqPrLWfUb2wGZDorAHz                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Not interested in this meeting                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: Done                                                                                              │
│    Args: {                                                                                                      │
│   "done": true                                                                                                  │
│ }                                                                                                               │
│    ID: toolu_01XC8968ks1RHk9bpu4euhbG                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ done=True                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Example 2: Simple Question

Test with a simple question that needs a direct response.

In [43]:
# Example email as markdown string
email_markdown_2 = """
**Subject**: LangGraph documentation link
**From**: bob@example.com
**To**: lance@langchain.dev

Hey Lance,

Could you send me the link to the LangGraph docs?

Thanks!
Bob

---
"""

# Display email with rich formatting
show_prompt(email_markdown_2, title="📧 Email to Process", border_style="cyan")

╭────────────────────────────────────────────── 📧 Email to Process ──────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  **Subject**: LangGraph documentation link                                                                      │
│  **From**: bob@example.com                                                                                      │
│  **To**: lance@langchain.dev                                                                                    │
│                                                                                                                 │
│  Hey Lance,                                                                                                     │
│                                                                                                                 │
│  Could you send me the link to the LangGraph docs?                                                              │
│                                                                                                                 │
│  Thanks!                                                                                                        │
│  Bob                                                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [44]:
config_2 = {"configurable": {"thread_id": "test-thread-2"}}

# Invoke agent with markdown email string
result_2 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_2}]},
    config=config_2,
)

# Display result with rich formatting
print("\n" + "="*60)
print("Agent Response")
print("="*60 + "\n")

# Check for interrupts (HITL)
if "__interrupt__" in result_2:
    print("🛑 INTERRUPT: Agent is waiting for your approval\n")
    
    for interrupt in result_2["__interrupt__"]:
        # New interrupt format with action_requests
        action_requests = interrupt.value.get('action_requests', [])
        review_configs = interrupt.value.get('review_configs', [])
        
        for action_req in action_requests:
            print("Action:", action_req['name'])
            print("Args:", action_req['args'])
            print("\nDescription:")
            show_prompt(action_req.get('description', ''), title="📋 Action Details", border_style="yellow")
            print("\n" + "-"*60)
        
        if review_configs:
            print("\nAllowed decisions:", review_configs[0].get('allowed_decisions', []))
    
    print("\n💡 To continue, you need to resume the agent with a decision.")
    print("   Use: resume_with_approve_or_reject(agent, config_2, decision_type='approve')")
else:
    format_messages(result_2["messages"])


Agent Response

🛑 INTERRUPT: Agent is waiting for your approval

Action: write_email
Args: {'to': 'bob@example.com', 'subject': 'Re: LangGraph documentation link', 'content': "Hi Bob,\n\nHere's the link to the LangGraph documentation: https://langchain-ai.github.io/langgraph/\n\nLet me know if you need anything else!\n\nBest,\nLance"}

Description:


╭─────────────────────────────────────────────── 📋 Action Details ───────────────────────────────────────────────╮
│                                                                                                                 │
│  I've drafted an email response. Please review the content, recipients, and subject line below. Approve to      │
│  send as-is, or Reject to cancel and end the workflow.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


------------------------------------------------------------

Allowed decisions: ['approve', 'reject']

💡 To continue, you need to resume the agent with a decision.
   Use: resume_with_approve_or_reject(agent, config_2, decision_type='approve')


## Test Memory Persistence

Test that memory persists across invocations using the same thread ID.

## Example 3: Triage Functionality

The agent now uses a triage tool to classify emails before taking action. This example shows how the agent triages a marketing email.

In [45]:
# Marketing email as markdown string
email_markdown_marketing = """
**Subject**: Limited Time Offer - 50% Off!
**From**: marketing@newsletter.com
**To**: lance@langchain.dev

Don't miss out on our amazing sale! Click here to shop now and save big on all your favorite items. This offer expires soon!

---
"""

# Display email
show_prompt(email_markdown_marketing, title="📧 Marketing Email", border_style="cyan")

# Invoke agent
config_marketing = {"configurable": {"thread_id": "test-marketing"}}
result_marketing = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_marketing}]},
    config=config_marketing,
)

# Show triage decision
print("\n" + "="*60)
print("Agent Decision")
print("="*60 + "\n")

messages = result_marketing.get("messages", [])
for i, msg in enumerate(messages):
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tool_call in msg.tool_calls:
            if tool_call.get('name') == 'triage_email':
                print("🔍 TRIAGE DECISION:")
                print(f"   Classification: {tool_call.get('args', {}).get('classification', 'N/A')}")
                print(f"   Reasoning: {tool_call.get('args', {}).get('reasoning', 'N/A')}")
                print()
    if hasattr(msg, 'name') and msg.name == 'triage_email':
        print(f"✓ Result: {msg.content}")
        print()

print("Since this was classified as 'ignore', the agent calls Done immediately without drafting a response.")

╭────────────────────────────────────────────── 📧 Marketing Email ───────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  **Subject**: Limited Time Offer - 50% Off!                                                                     │
│  **From**: marketing@newsletter.com                                                                             │
│  **To**: lance@langchain.dev                                                                                    │
│                                                                                                                 │
│  Don't miss out on our amazing sale! Click here to shop now and save big on all your favorite items. This       │
│  offer expires soon!                                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Agent Decision

🔍 TRIAGE DECISION:
   Classification: ignore
   Reasoning: This email is from marketing@newsletter.com with the subject "Limited Time Offer - 50% Off!" and contains promotional content about a sale. According to Lance's user profile, marketing newsletters and promotional emails should be classified as 'ignore'. This is clearly a marketing/promotional email with no direct questions, no technical work discussion, and no actionable items relevant to Lance's work at LangChain.

✓ Result: Classification Decision: ignore. Reasoning: This email is from marketing@newsletter.com with the subject "Limited Time Offer - 50% Off!" and contains promotional content about a sale. According to Lance's user profile, marketing newsletters and promotional emails should be classified as 'ignore'. This is clearly a marketing/promotional email with no direct questions, no technical work discussion, and no actionable items relevant to Lance's work at LangChain.

Since this was classified as '

In [ ]:
# Follow-up email as markdown string
email_markdown_3 = """
**Subject**: Follow-up meeting
**From**: jane@example.com
**To**: lance@langchain.dev

Hi again,

Can we also schedule a follow-up meeting next week?

Jane

---
"""

# Use same config as example 1 (same thread_id) to test memory persistence
result_3 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_3}]},
    config=config_1,  # Same thread as example 1
)

# Display result with rich formatting (show last 5 messages)
print("\n" + "="*60)
print("Agent Response (with memory from previous conversation)")
print("="*60 + "\n")
format_messages(result_3["messages"][-5:])

## Test Memory Updates on Rejection

Test that rejecting tool calls updates the user profile in the store.

In [46]:
print("✓ Memory testing section - helper functions already loaded above")

✓ Memory testing section - helper functions already loaded above


### Step 1: Check Initial User Profile

Before any rejections, let's see the default user profile.

In [47]:
# Get initial user profile
initial_profile = get_user_profile_from_store(agent)

print("📋 Initial User Profile:")
print("="*60)
if initial_profile:
    print(initial_profile)
else:
    print("(No profile in store yet - will use default on first use)")
print("="*60)

📋 Initial User Profile:
I'm Lance, a software engineer at LangChain. When triaging emails:
- Respond to: Direct questions about technical work, project inquiries, family/self-care reminders
- Notify: GitHub notifications, deadline reminders, FYI project updates, team status messages, meeting requests about project roadmap discussions
- Ignore: Marketing newsletters, promotional emails, CC'd FYI threads

Response style:
- Professional and concise tone
- Acknowledge deadlines explicitly in responses
- For technical questions: State investigation approach and timeline
- For event invitations: Ask about workshops, discounts, and deadlines; don't commit immediately
- For collaboration requests: Acknowledge existing materials, mention reviewing them

Meeting scheduling:
- Prefer 30-minute meetings (15 minutes acceptable)
- If times proposed: Verify all options, commit to one or decline
- If no times proposed: Offer multiple options instead of picking one
- Reference meeting purpose and durat

### Step 2: Send Newsletter Email and Reject Draft

Send a newsletter email that the agent will try to respond to, then reject it to update memory.

In [49]:
# Email that agent might try to respond to
meeting_email = """
**Subject**: Sync on agent architecture
**From**: Alex Rodriguez <alex.rodriguez@company.com>
**To**: Lance Martin <lance@langchain.dev>

Hey Lance,

Can we schedule 30 minutes next week to discuss the multi-agent architecture for our project? I have some questions about routing between agents and want to get your input.

I'm free Tuesday afternoon or Thursday morning. Let me know what works!

Best,
Alex
"""

show_prompt(meeting_email, title="📧 Meeting Email", border_style="cyan")

# Use a new thread for this test
config_reject_test = {"configurable": {"thread_id": "test-reject-newsletter"}}

# Invoke agent
result_newsletter = agent.invoke(
    {"messages": [{"role": "user", "content": meeting_email}]},
    config=config_reject_test,
)

# Check if interrupted
if "__interrupt__" in result_newsletter:
    print("\n🛑 INTERRUPT: Agent wants to respond to this meeting\n")
    
    for interrupt in result_newsletter["__interrupt__"]:
        # New interrupt format with action_requests
        action_requests = interrupt.value.get('action_requests', [])
        
        for action_req in action_requests:
            print("Action:", action_req['name'])
            print("Args:", action_req['args'])
            print("\n" + "-"*60)
else:
    print("\n✓ Agent classified this as ignore/notify (no response needed)")
    format_messages(result_newsletter["messages"][-3:])

╭─────────────────────────────────────────────── 📧 Meeting Email ────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  **Subject**: Sync on agent architecture                                                                        │
│  **From**: Alex Rodriguez <alex.rodriguez@company.com>                                                          │
│  **To**: Lance Martin <lance@langchain.dev>                                                                     │
│                                                                                                                 │
│  Hey Lance,                                                                                                     │
│                                                                                                                 │
│  Can we schedule 30 minutes next week to discuss the multi-agent architecture for our project? I have some      │
│  questions about routing between agents and want to get your input.                                             │
│                                                                                                                 │
│  I'm free Tuesday afternoon or Thursday morning. Let me know what works!                                        │
│                                                                                                                 │
│  Best,                                                                                                          │
│  Alex                                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🛑 INTERRUPT: Agent wants to respond to this meeting

Action: write_email
Args: {'to': 'alex.rodriguez@company.com', 'subject': 'Re: Sync on agent architecture', 'content': "Hey Alex,\n\nHappy to discuss the multi-agent architecture and routing questions with you. I have availability during the times you mentioned:\n\n- Tuesday, December 10: 2:00 PM or 4:00 PM\n- Thursday, December 12: 9:00 AM\n\nLet me know which works best for you and I'll send a calendar invite for 30 minutes.\n\nBest,\nLance"}

------------------------------------------------------------


### Step 3: Reject with Feedback

Reject the draft and provide feedback about why newsletters shouldn't be responded to.

In [50]:
# Reject with feedback
rejection_message = "Let's not respond to emails requesting meetings from any @company.com email addresses."

print("🚫 Rejecting with feedback:", rejection_message)
print()

result_rejected = resume_with_approve_or_reject(
    agent, 
    config_reject_test, 
    decision_type="reject",
    message=rejection_message
)

print("\n✓ Rejection processed")
format_messages(result_rejected["messages"][-2:])

# Verify the rejection message was captured
print("\n🔍 Checking rejection message in tool messages...")
for msg in result_rejected["messages"]:
    if hasattr(msg, "status") and msg.status == "error":
        print(f"   ToolMessage with status='error' found:")
        print(f"   - Name: {msg.name}")
        print(f"   - Content: {msg.content}")
        print(f"   ✓ Rejection message captured correctly!")

🚫 Rejecting with feedback: Let's not respond to emails requesting meetings from any @company.com email addresses.


✓ Rejection processed


╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ done=True                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Understood! I've stopped the workflow as requested. No response will be sent to this meeting request from the   │
│ @company.com address.                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🔍 Checking rejection message in tool messages...
   ToolMessage with status='error' found:
   - Name: write_email
   - Content: Let's not respond to emails requesting meetings from any @company.com email addresses.
   ✓ Rejection message captured correctly!


### Step 4: Check Updated User Profile

The profile should now include guidance about newsletters based on the rejection.

In [51]:
# Get updated user profile
updated_profile = get_user_profile_from_store(agent)

print("📋 Updated User Profile After Rejection:")
print("="*60)
if updated_profile:
    print(updated_profile)
    
    # Highlight the differences
    print("\n" + "="*60)
    print("🔍 Changes from rejection:")
    print("   - Should now include guidance about newsletters")
    print("   - Likely updated 'Notify' or 'Ignore' section")
else:
    print("(No profile found)")
print("="*60)

📋 Updated User Profile After Rejection:
I'm Lance, a software engineer at LangChain. When triaging emails:
- Respond to: Direct questions about technical work, project inquiries, family/self-care reminders
- Notify: GitHub notifications, deadline reminders, FYI project updates, team status messages, meeting requests about project roadmap discussions, meeting requests from @company.com addresses
- Ignore: Marketing newsletters, promotional emails, CC'd FYI threads

Response style:
- Professional and concise tone
- Acknowledge deadlines explicitly in responses
- For technical questions: State investigation approach and timeline
- For event invitations: Ask about workshops, discounts, and deadlines; don't commit immediately
- For collaboration requests: Acknowledge existing materials, mention reviewing them

Meeting scheduling:
- Prefer 30-minute meetings (15 minutes acceptable)
- If times proposed: Verify all options, commit to one or decline
- If no times proposed: Offer multiple option

We now see `meeting requests from @company.com addresses` added to `ignore`.